In [ ]:
import cv2
import numpy as np
from PIL import Image
import os
from sklearn.cluster import KMeans

In [ ]:
import zipfile

# Path to the zip file
zip_filename = '/content/images.zip'  # Replace with the path to your zip file

# Path to the directory where you want to extract the files
extract_to = '/content/images'  # Replace with the path where you want to extract the files

# Create the directory if it doesn't exist
os.makedirs(extract_to, exist_ok=True)

# Unzip the file
with zipfile.ZipFile(zip_filename, 'r') as zip_ref:
    zip_ref.extractall(extract_to)

print(f'Files have been extracted to {extract_to}')

Files have been extracted to /content/images


# Data Preparation

In [ ]:
def isolate_and_fill_circles(image_path, save_path, fixed_radius=20):
    # Load image
    img = cv2.imread(image_path, cv2.IMREAD_UNCHANGED)

    # Define the color ranges for each circle in HSV format
    color_ranges = {
        "purple": ([140, 50, 50], [160, 255, 255]),
        "blue": ([90, 50, 50], [130, 255, 255]),
        "orange": ([10, 100, 100], [25, 255, 255]),
        "green": ([40, 50, 50], [80, 255, 255])
    }

    # Define the output colors (in BGR format)
    output_colors = {
        "purple": (190, 74, 199, 255),
        "blue": (0, 181, 236, 255),
        "orange": (251, 176, 59, 255),
        "green": (115, 210, 22, 255)
    }

    # Convert the image to HSV color space
    hsv_img = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)

    # Create an empty image for results with a transparent background
    result_img = np.zeros((img.shape[0], img.shape[1], 4), dtype=np.uint8)

    for color_name, (lower, upper) in color_ranges.items():
        lower = np.array(lower, dtype="uint8")
        upper = np.array(upper, dtype="uint8")
        color_mask = cv2.inRange(hsv_img, lower, upper)

        # Detect circles using HoughCircles
        circles = cv2.HoughCircles(color_mask, cv2.HOUGH_GRADIENT, dp=1.2, minDist=20, param1=50, param2=30, minRadius=5, maxRadius=30)

        if circles is not None:
            circles = np.round(circles[0, :]).astype("int")
            for (x, y, radius) in circles:
                # Draw the filled circle with the mapped output color and fixed radius
                cv2.circle(result_img, (x, y), fixed_radius, output_colors[color_name], -1)

    # Save the new image
    isolated_img_pil = Image.fromarray(result_img)
    isolated_img_pil.save(save_path)

def resize_image(input_path, output_path, size=(128, 128)):
    # Open the image file
    img = Image.open(input_path)

    # Resize the image
    resized_img = img.resize(size, Image.ANTIALIAS)

    # Save the resized image
    resized_img.save(output_path)

def remove_low_opacity_pixels(image_path, output_path, opacity_threshold=180):
    # Open the image
    image = Image.open(image_path).convert("RGBA")

    # Get image data
    data = image.getdata()

    # Process the pixels
    new_data = []
    for item in data:
        # Change all pixels below the threshold to fully transparent
        if item[3] < opacity_threshold:
            new_data.append((255, 255, 255, 0))  # White color with 0% opacity (fully transparent)
        else:
            new_data.append(item)

    # Update image data
    image.putdata(new_data)

    # Save the image
    image.save(output_path)

In [ ]:
def process_batch(input_dir, output_dir, processed_dir, opacity_processed_dir, size=(128, 128), fixed_radius=20):
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)
    if not os.path.exists(processed_dir):
        os.makedirs(processed_dir)
    if not os.path.exists(opacity_processed_dir):
        os.makedirs(opacity_processed_dir)

    for filename in os.listdir(input_dir):
        if filename.endswith('.png'):
            input_path = os.path.join(input_dir, filename)
            output_path = os.path.join(output_dir, filename)
            resized_output_path = os.path.join(processed_dir, filename)
            opacity_output_path = os.path.join(opacity_processed_dir, filename)

            # Process the image to isolate and fill circles
            isolate_and_fill_circles(input_path, output_path, fixed_radius=fixed_radius)

            # Resize the processed image
            resize_image(output_path, resized_output_path, size=size)

            # Opacity filter the processed image
            remove_low_opacity_pixels(resized_output_path, opacity_output_path)

input_dir = '/content/images'
output_dir = '/content/Output'
processed_dir = '/content/Processed'
opacity_processed_dir = '/content/Opacity'

process_batch(input_dir, output_dir, processed_dir, opacity_processed_dir, size=(128, 128), fixed_radius=20)

<ipython-input-3-5cbb264abf84>:50: DeprecationWarning: ANTIALIAS is deprecated and will be removed in Pillow 10 (2023-07-01). Use LANCZOS or Resampling.LANCZOS instead.
  resized_img = img.resize(size, Image.ANTIALIAS)
